In [10]:

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from joblib import dump
import optuna
from optuna.integration import LightGBMPruningCallback
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import optuna
from optuna.integration import LightGBMPruningCallback
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import joblib
import optuna.visualization as vis
from sklearn.model_selection import cross_val_score, KFold
import warnings
warnings.filterwarnings("ignore")

In [7]:
# データの読み込み
train_data = pd.read_csv('../data_processed/train_data_processed.csv')
test_data = pd.read_csv('../data_processed/test_data_processed.csv')
submit_data = pd.read_csv('../data/test.csv')

X = train_data.drop(['attendance'], axis=1)
y = train_data['attendance']

# データの分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# 目的関数の定義
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 20),  # max_depthを1から20の範囲に拡大
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.00001, 1.0),  # learning_rateを0.00001から1.0の範囲に拡大
        'n_estimators': trial.suggest_int('n_estimators', 50, 2000),  # n_estimatorsを50から2000の範囲に拡大
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),  # min_child_weightを1から20の範囲に拡大
        'gamma': trial.suggest_discrete_uniform('gamma', 0.1, 5, 0.1),  # gammaを0.1から5の範囲に拡大
        'subsample': trial.suggest_discrete_uniform('subsample', 0.5, 1.0, 0.05),  # subsampleを0.5から1.0の範囲に拡大
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1.0, 0.05),  # colsample_bytreeを0.5から1.0の範囲に拡大
        'reg_alpha': trial.suggest_discrete_uniform('reg_alpha', 0.0, 1.0, 0.1),  # reg_alphaを0.0から1.0の範囲に拡大
        'reg_lambda': trial.suggest_discrete_uniform('reg_lambda', 0.0, 1.0, 0.1)  # reg_lambdaを0.0から1.0の範囲に拡大
    }
    model = xgb.XGBRegressor(**params, random_state=42)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=False,
              callbacks=[XGBoostPruningCallback(trial, 'validation_0-rmse')])
    preds = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    return rmse

# 最適化の実行
study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner(n_warmup_steps=10))
study.optimize(objective, n_trials=500, timeout=1800)  # n_trialsを200に増やし、timeoutを7200秒に増やす

# 最適なパラメータの出力
print('Best trial:', study.best_trial.params)

[I 2023-06-29 03:38:17,185] A new study created in memory with name: no-name-ae0c17a7-fb43-4e8f-9e2c-8f1552694f0c
[I 2023-06-29 03:38:22,873] Trial 0 finished with value: 19167.781025277473 and parameters: {'max_depth': 6, 'learning_rate': 3.13217699803811e-05, 'n_estimators': 1319, 'min_child_weight': 17, 'gamma': 4.7, 'subsample': 0.8500000000000001, 'colsample_bytree': 0.95, 'reg_alpha': 0.2, 'reg_lambda': 0.0}. Best is trial 0 with value: 19167.781025277473.
[I 2023-06-29 03:38:30,206] Trial 1 finished with value: 18984.11079443262 and parameters: {'max_depth': 16, 'learning_rate': 4.0389483661712456e-05, 'n_estimators': 1293, 'min_child_weight': 11, 'gamma': 0.4, 'subsample': 0.5, 'colsample_bytree': 0.8, 'reg_alpha': 0.6000000000000001, 'reg_lambda': 0.30000000000000004}. Best is trial 1 with value: 18984.11079443262.
[I 2023-06-29 03:38:36,938] Trial 2 finished with value: 5327.714062169808 and parameters: {'max_depth': 9, 'learning_rate': 0.0013648970819363038, 'n_estimators': 

Best trial: {'max_depth': 6, 'learning_rate': 0.1341702126383532, 'n_estimators': 551, 'min_child_weight': 10, 'gamma': 4.3999999999999995, 'subsample': 0.95, 'colsample_bytree': 0.55, 'reg_alpha': 0.2, 'reg_lambda': 0.7000000000000001}


In [16]:
# 学習曲線の描画
vis.plot_optimization_history(study)

In [17]:
# ハイパーパラメータ探索空間の視覚化
vis.plot_parallel_coordinate(study)

In [19]:
# 最適なモデルの取得
best_params = study.best_trial.params
best_model = xgb.XGBRegressor(**best_params, random_state=42)
best_model.fit(X, y)

# モデルを保存
best_model.save_model('../models/XGBoost')